In [21]:
%run ./pre_processing.ipynb
import pandas as pd
from icecream import ic
# ic.disable()
folds: list[pd.DataFrame]
folds

[      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
 0               7.6              0.32         0.58           16.75      0.050   
 1               5.7              0.24         0.47            6.30      0.069   
 2               7.6              0.36         0.44            8.30      0.255   
 3               7.4              0.47         0.46            2.20      0.114   
 4               9.9              0.49         0.23            2.40      0.087   
 ...             ...               ...          ...             ...        ...   
 1294            6.9              0.38         0.25            9.80      0.040   
 1295            7.3              0.23         0.37            1.80      0.032   
 1296            6.6              0.36         0.24            0.90      0.038   
 1297            6.4              0.64         0.21            1.80      0.081   
 1298            7.1              0.52         0.03            2.60      0.076   
 
       free su

In [7]:
from algos.baseline import Baseline
from sklearn.metrics import mean_absolute_error as MAE, accuracy_score as Accuracy
from imblearn.metrics import macro_averaged_mean_absolute_error as MAMAE
from algos.CEM import CEM
import numpy as np
metrics: list[dict[str, float]] = []

for i in range(len(folds)):
  train_cv, test_cv = pd.concat(folds[j] for j in range(len(folds)) if j != i), folds[i]
  baseline = Baseline()
  baseline.fit(train_cv.drop("quality", axis=1), train_cv["quality"])
  y_pred = baseline.predict(test_cv.drop("quality", axis=1))
  metrics.append({
    "MAE": MAE(test_cv["quality"], y_pred),
    "Accuracy": Accuracy(test_cv["quality"], np.round(y_pred)),
    "MAMAE": MAMAE(test_cv["quality"], np.round(y_pred)),
    "CEM": CEM(test_cv["quality"], np.round(y_pred))
  })

{metric: f"{val} +- {pd.DataFrame(metrics).std()[metric]}" for metric, val in pd.DataFrame(metrics).mean().items()}

{'MAE': '0.6372121750458933 +- 0.01567434485868058',
 'Accuracy': '0.43651151773553626 +- 0.008160939902022954',
 'MAMAE': '1.6714285714285715 +- 0.09583148474999095',
 'CEM': '0.13912233473619084 +- 0.3110869976586029'}

In [22]:
import sys
from importlib import reload
reload(sys.modules.get('algos.CART', sys))
reload(sys.modules.get('algos.CEM', sys))
from algos.CEM import CEM
from algos.CART import CART
import numpy as np
metrics: list[dict[str, float]] = []

for i in range(len(folds)):
  train_cv, test_cv = pd.concat(folds[j] for j in range(len(folds)) if j != i), folds[i]
  cart = CART()
  cart.fit(np.array(train_cv.drop("quality", axis=1)), train_cv["quality"].values)
  y_pred = cart.predict(test_cv.drop("quality", axis=1).values)
  metrics.append({
    "MAE": MAE(test_cv["quality"], y_pred),
    "Accuracy": Accuracy(test_cv["quality"], np.round(y_pred)),
    "MAMAE": MAMAE(test_cv["quality"], np.round(y_pred)),
    "CEM": CEM(test_cv["quality"], np.round(y_pred))
  })

{metric: f"{val} +- {pd.DataFrame(metrics).std()[metric]}" for metric, val in pd.DataFrame(metrics).mean().items()}

{'MAE': '0.5558070566532147 +- 0.0168574048165228',
 'Accuracy': '0.5530249304198496 +- 0.007474081528998565',
 'MAMAE': '1.290245134298197 +- 0.17210055806053776',
 'CEM': '0.837566694833653 +- 0.009441048106303671'}

In [23]:
reload(sys.modules.get('algos.MLR_GR', sys))
from algos.MLR_GR import MLR_GR

metrics: list[dict[str, float]] = []
for i in range(len(folds)):
  train_cv, test_cv = pd.concat(folds[j] for j in range(len(folds)) if j != i), folds[i]
  mlr = MLR_GR()
  mlr.fit(np.array(train_cv.drop("quality", axis=1)), train_cv["quality"].values)
  y_pred = mlr.predict(test_cv.drop("quality", axis=1).values)
  ic(test_cv['quality'])
  metrics.append({
    "MAE": MAE(*ic(test_cv["quality"], y_pred)),
    "Accuracy": Accuracy(test_cv["quality"], np.round(y_pred)),
    "MAMAE": MAMAE(test_cv["quality"].to_numpy(), ic(np.round(ic(y_pred)))),
    "CEM": CEM(test_cv["quality"], np.round(y_pred))
  })

{metric: f"{val} +- {pd.DataFrame(metrics).std()[metric]}" for metric, val in pd.DataFrame(metrics).mean().items()}

ic| X: array([[ 0.29426685, -0.12497996,  1.8053622 , ...,  0.04955994,
               -1.09002623, -0.57809075],
              [-1.1723451 , -0.61053765,  1.04754374, ..., -0.48297245,
               -0.641715  , -0.57809075],
              [ 0.29426685,  0.11779888,  0.84086597, ..., -0.6826721 ,
               -0.24944266, -0.57809075],
              ...,
              [-0.47763418,  0.11779888, -0.53698578, ..., -0.94893829,
                0.42302419, -0.57809075],
              [-0.63201438,  1.8172508 , -0.74366354, ...,  0.84835852,
               -0.58567609,  1.72983222],
              [-0.09168367,  1.08891427, -1.98373012, ...,  0.44895923,
               -0.58567609,  1.72983222]], shape=(1299, 12))
    self.theta: array([ 0.041389  , -0.25131941, -0.01203265,  0.2031117 , -0.02594886,
                        0.09636871, -0.0908701 , -0.15094749,  0.03439877,  0.1007179 ,
                        0.34343149,  0.10739375])
ic| test_cv['quality']: 0       5
                  

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [2]:
import sys
from importlib import reload
reload(sys.modules.get('algos.k_rank', sys))
from algos.k_rank import KRank
import numpy as np

metrics: list[dict[str, float]] = []
for i in range(len(folds)):
    train_cv, test_cv = pd.concat(folds[j] for j in range(len(folds)) if j != i), folds[i]
    krank = KRank()
    krank.fit(np.array(train_cv.drop("quality", axis=1).values, dtype=np.float32), train_cv["quality"].values)
    y_pred = pd.Series(krank.predict(test_cv.drop("quality", axis=1).values))
    metrics.append({
        "MAE": MAE(test_cv["quality"], y_pred),
        "Accuracy": Accuracy(test_cv["quality"], y_pred),
        "MAMAE": MAMAE(test_cv["quality"], y_pred),
        "CEM": CEM(test_cv["quality"], y_pred),
    })
{metric: f"{val} +- {pd.DataFrame(metrics).std()[metric]}" for metric, val in pd.DataFrame(metrics).mean().items()}

NameError: name 'MAE' is not defined